In [1]:
from scipy.stats import invgamma
from scipy.stats import norm
from multiprocessing import Pool
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

In [5]:
def integral(estimate, ts):
    elements = np.ones(len(ts) - 1)
    for i in range(len(ts) - 1):
        elements[i] = (ts[i+1] - ts[i])*(estimate[i+1] + estimate[i])/2
    return sum(elements)

def sum1(beta):
    elements = np.ones(N)
    for i in range(N):
        elements[i] = p[i, 0] - beta * (p[i, 1] - ave_x)
    return np.sum(elements)

def sum2(alpha):
    elements = np.ones(N)
    for i in range(N):
        elements[i] = (p[i, 1] - ave_x)*(p[i, 0] - alpha)
    return np.sum(elements)

def sum3(alpha, beta):
    elements = np.ones(N)
    for i in range(N):
        elements[i] = (p[i, 0] - alpha - beta*(p[i, 1] - ave_x))**2
    return np.sum(elements)

def loglike(alpha, beta, sigma):
    return N*np.log(1/(sigma*np.sqrt(2*np.pi))) - sum3(alpha, beta) / (2*sigma)

def function1(w):
    if w < 100:
        np.random.seed(datetime.now().microsecond)
        
        ###### ここを変えてね！
        n = 10
        c = 2
        ######
        
        estimates = np.ones(n+1)
        ts = np.ones(n+1)

        for i in range(n+1):
            t = (i/n)**c
            ts[i] = t

            alphas = np.ones(sample_iter)
            betas = np.ones(sample_iter)
            sigmas = np.ones(sample_iter)
            
            if i == 0:
                alphas[0] = 3000
                betas[0] = 185
                sigmas[0] = 90000

            else:
                alphas[0] = np.mean(alpha_sample).item()
                betas[0] = np.mean(beta_sample).item()
                sigmas[0] = np.mean(sigma_sample).item()

            for j in range(sample_iter -1):

                location_alpha = (sigma_alpha*t*sum1(betas[-1]) + sigmas[-1]*mu_alpha) / (sigma_alpha * N*t + sigmas[-1])
                scale_alpha = np.sqrt((sigma_alpha * sigmas[-1]) / (sigma_alpha * N*t + sigmas[-1]))
                r = norm.rvs(loc = location_alpha, scale = scale_alpha)
                alphas[j+1] = r.item()

                location_beta = (sigma_beta * t * sum2(alphas[-1]) + sigmas[-1] * mu_beta) / (sigma_beta * var*N + sigmas[-1])
                scale_beta = (sigmas[-1] * sigma_beta) / (sigma_beta * var*N + sigmas[-1])
                q = norm.rvs(loc = location_beta, scale = scale_beta)
                betas[j+1] = q.item()

                shape = N*t/2 + a
                invrate = 2*b / (b*t*sum3(alphas[-1], betas[-1]) + 2)
                s = invgamma.rvs(a = shape, scale = 1/ invrate)
                sigmas[j+1] = s.item()

            alpha_sample = alphas[burn_in:]
            beta_sample = betas[burn_in:len(betas)]
            sigma_sample = sigmas[burn_in:len(sigmas)]

            box = np.ones(len(alpha_sample))
            for k, l in enumerate(alpha_sample):
                box[k] = loglike(l, beta_sample[k], sigma_sample[k])

            estimates[i] = np.average(box)
        
        return estimates
    
def sum4(beta):
    elements = np.ones(N)
    for i in range(N):
        elements[i] = p[i, 0] - beta * (p[i, 2] - ave_z)
    return np.sum(elements)

def sum5(alpha):
    elements = np.ones(N)
    for i in range(N):
        elements[i] = (p[i, 2] - ave_z)*(p[i, 0] - alpha)
    return np.sum(elements)

def sum6(alpha, beta):
    elements = np.ones(N)
    for i in range(N):
        elements[i] = (p[i, 0] - alpha - beta*(p[i, 2] - ave_z))**2
    return np.sum(elements)    
    
def function2(w):
    if w < 100:
        np.random.seed(datetime.now().microsecond)
        
        ###### ここを変えてね！
        n = 10
        c = 2
        ######
        
        estimates = np.ones(n+1)
        ts = np.ones(n+1)

        for i in range(n+1):
            t = (i/n)**c
            ts[i] = t

            alphas = np.ones(sample_iter)
            betas = np.ones(sample_iter)
            sigmas = np.ones(sample_iter)
            
            if i == 0:
                alphas[0] = 3000
                betas[0] = 185
                sigmas[0] = 90000

            else:
                alphas[0] = np.mean(alpha_sample).item()
                betas[0] = np.mean(beta_sample).item()
                sigmas[0] = np.mean(sigma_sample).item()

            for j in range(sample_iter - 1):

                location_alpha = (sigma_alpha*t*sum4(betas[-1]) + sigmas[-1]*mu_alpha) / (sigma_alpha * N*t + sigmas[-1])
                scale_alpha = np.sqrt((sigma_alpha * sigmas[-1]) / (sigma_alpha * N*t + sigmas[-1]))
                r = norm.rvs(loc = location_alpha, scale = scale_alpha)
                alphas[j+1] = r.item()

                location_beta = (sigma_beta * t * sum5(alphas[-1]) + sigmas[-1] * mu_beta) / (sigma_beta * var*N + sigmas[-1])
                scale_beta = (sigmas[-1] * sigma_beta) / (sigma_beta * var2*N + sigmas[-1])
                q = norm.rvs(loc = location_beta, scale = scale_beta)
                betas[j+1] = q.item()

                shape = N*t/2 + a
                invrate = 2*b / (b*t*sum6(alphas[-1], betas[-1]) + 2)
                s = invgamma.rvs(a = shape, scale = 1/ invrate)
                sigmas[j+1] = s.item()

            alpha_sample = alphas[burn_in:]
            beta_sample = betas[burn_in:len(betas)]
            sigma_sample = sigmas[burn_in:len(sigmas)]

            box = np.ones(len(alpha_sample))
            for k, l in enumerate(alpha_sample):
                box[k] = loglike(l, beta_sample[k], sigma_sample[k])

            estimates[i] = np.average(box)
        
        return estimates

In [6]:
pine = pd.read_table("pine.txt", delim_whitespace = True)
p = pine.values
ave_x = np.average(p[:, 1]).item()
ave_z = np.average(p[:, 2]).item()

mu_alpha = 3000
sigma_alpha = 10**6
mu_beta = 185
sigma_beta = 10**4
a = 3
b = 1/(2*300**2)

N = np.shape(p)[0]
var = np.var(p[:, 1])
var2 = np.var(p[:, 2])

# ここをいじってね。
sample_iter = 1000
burn_in = 30
m = 10
core = 2


model1

In [7]:
if __name__ == '__main__':
    p = Pool(core) 
    result1 = p.map(function1, range(m))

In [8]:
result1

[array([-1162.52902715,  -633.41611496,  -663.48690142,  -682.3470266 ,
         -693.4285553 ,  -699.94416682,  -702.95756019,  -705.40559998,
         -707.81619989,  -708.29763219,  -709.90599426]),
 array([-1124.52958724,  -633.87968423,  -664.60981538,  -683.09516496,
         -692.60754662,  -699.9427733 ,  -703.36268927,  -705.4368735 ,
         -706.79137928,  -708.42717889,  -709.74004089]),
 array([-1119.21088245,  -633.10268767,  -663.76120352,  -682.72688064,
         -692.89936497,  -699.83136087,  -703.22172676,  -706.64662504,
         -707.29813989,  -708.87279634,  -709.77371605]),
 array([-1158.15919133,  -633.97548807,  -664.1311928 ,  -683.4136767 ,
         -693.35033945,  -699.77643764,  -703.6454459 ,  -706.13991865,
         -707.35764503,  -708.60570005,  -709.65357203]),
 array([-1113.31313777,  -633.24663448,  -664.65744344,  -682.54166454,
         -693.35939285,  -700.25090746,  -703.3132207 ,  -706.65327559,
         -707.90979701,  -708.52598532,  -709.89

In [30]:
ts = [(i/10)**2 for i in range(11)]

expect1 = np.ones(10)
for i in range(10):
    expect1[i] = integral(result1[i], ts)

In [31]:
expect1

array([-884.188028  , -886.39347799, -885.99979868, -886.94892566,
       -882.79827067, -883.56345496, -884.75868616, -883.08989153,
       -884.89411503, -887.26589149])

model2

In [9]:
if __name__ == '__main__':
    result2 = p.map(function2, range(m))

In [33]:
ts = [(i/10)**2 for i in range(11)]

expect2 = np.ones(10)
for i in range(10):
    expect2[i] = integral(result2[i], ts)

In [34]:
expect2

array([-884.16981721, -883.15487573, -880.10643773, -888.0550601 ,
       -882.85498881, -881.53405693, -881.25910846, -879.96780886,
       -880.71796384, -882.66026067])

BF21

In [35]:
bf_21 = []
for a,b in zip(expect1, expect2):
    bf_21.append(np.exp(b-a))

In [36]:
bf_21

[1.0183776157528717,
 25.498057362920129,
 362.62199410646258,
 0.33083535789278612,
 0.94486034842332289,
 7.6095043086417604,
 33.101470302187131,
 22.693593743229702,
 65.114755769604017,
 100.04607451802922]